In [1]:
import pandas as pd
import numpy as np

In [4]:
# DataFrames
dinosaur = pd.read_csv('Datasets/jurassicparkwithweights.csv')
distances = pd.read_csv('Datasets/distances.csv')
animal = pd.read_csv('Datasets/Clean-Animal-Info.csv')

In [5]:
# Clean distances DataFrame
def clean_whitespace(country):
    return country[1:]

distances = distances.assign(Country2=distances['Country2'].apply(clean_whitespace))
distances = distances.set_index('Country1')

In [6]:
# Clean dinosaur DataFrame
def format_diet(diet): 
    conversion = {
        "herbivorous": 'Herbivore',
        "carnivorous": 'Carnivore',
        "omnivorous": 'Omnivore'
    }
    if diet in conversion:
        return conversion[diet]
    return None

def format_height(height):
    height = str(height)
    height = height.strip('m')
    return .53 * float(height)

def clean_countries(country):
    if country == 'USA':
        return "United States"
    elif country == 'North Africa':
        return "Algeria"
    elif country == 'Wales':
        return "United Kingdom"
    else:
        return country
    
def fix_diet(diet):
    insectivore = diet == 'Insectivore' or diet == 'Carnivore, Insectivore'
    piscivore = diet == 'Carnivore, Piscivore' or diet == 'Piscivore'
    scavenger = diet == 'Carnivore, Scavenger'
    filter_feeder = diet == 'Filter Feeder'
    omnivore = diet == 'Herbivore, Omnivore' or diet == 'Omnivore, Herbivore'
    insects = diet == 'Nectar, Insects' or diet == 'Insectivore, Herbivore' or diet == 'Omnivore, Insectivore'
    
    if insectivore or piscivore or scavenger or filter_feeder:
        return 'Carnivore'
    elif omnivore or insects:
        return 'Omnivore'
    else:
        return diet 

def convert_diet(diet):
    if diet == 'Carnivore':
        return 0
    elif diet == 'Omnivore':
        return 1
    elif diet == 'Herbivore':
        return 2    

new_dino = dinosaur.assign(
    Diet = dinosaur['diet'].apply(format_diet),
    Country = dinosaur['lived_in'],
    Height = dinosaur['length'].apply(format_height)
)

final_dino = new_dino[['name', 'Diet','Country', 'Height', 'weight']]
final_dino = final_dino.set_index('name')
final_dino = final_dino.assign(Country=final_dino['Country'].apply(clean_countries))
final_dino = final_dino.dropna()
final_dino = final_dino.assign(Diet=final_dino['Diet'].apply(convert_diet))

In [7]:
# Clean animal DataFrame
def convert_height(height): 
    return height / 100

final_animal = animal[['Height', 'Diet', 'Animal', 'Countries Found', 'Weight']]

final_animal = final_animal.assign(
    Height = final_animal['Height'].apply(convert_height),
    Country = final_animal['Countries Found']
)

final_animal = final_animal.set_index('Animal')

final_animal = final_animal[['Diet', 'Country','Height', 'Weight']]
final_animal = final_animal.assign(Diet=final_animal['Diet'].apply(fix_diet))
final_animal = final_animal.assign(Diet=final_animal['Diet'].apply(convert_diet))

In [8]:
# Normalization
def normalize(x, m):
    return (0.5*m) / (1+np.exp(10/m * (0.5*m-x)))

def find_difference(animal, feature1, feature2, index):
    difference = np.abs(final_animal.loc[animal][feature1]-final_dino[feature2])
    feature_mean = difference.mean()
    delta_feature = np.abs((final_animal.loc[animal][feature1] - final_dino.iloc[index][feature2]))
    out = ((delta_feature) / feature_mean) ** 2
    return out

In [9]:
# Similarity Function
def calculate_similarity(animal):
    similarity_array = np.array([])
    animal_location = final_animal.loc[animal]['Country']
    distances_location = distances.reset_index()
    distances_location = distances_location[distances_location['Country1'] == animal_location]
    
    for i in range(final_dino.shape[0]):
        dist_sum = 0
        #Diet Similarity
        difference = np.abs(final_animal.loc[animal]['Diet']-final_dino['Diet'])
        dist_sum += difference.iloc[i] ** 2
             
        #Height Similarity
        dist_sum += find_difference(animal, 'Height', 'Height', i)
        
        
        #Weight Similarity
        dist_sum += find_difference(animal, 'Weight', 'weight', i)
        
        #Distance Similarity
        delta_dist = distances_location[distances_location['Country2']==final_dino.iloc[i]['Country']]['Distance']
        dist_mean = distances_location['Distance'].mean()

        dist_sum += ((delta_dist) / dist_mean) ** 2
        
        similarity_score = dist_sum ** 0.5
        similarity_array = np.append(similarity_array, similarity_score)
    
    updated_df = final_dino.assign(Similarity_Score=similarity_array)
    return updated_df.sort_values('Similarity_Score', ascending=True)

test = calculate_similarity('Zebra')
test


ValueError: Length of values (0) does not match length of index (283)

In [10]:
distances_locationtest[distances_locationtest['Country2']==final_dino.iloc[1]['Country']]['Distance']

NameError: name 'distances_locationtest' is not defined

In [11]:
dinotest = final_dino.reset_index()
dinotest[dinotest['name'] == 'shanag']

,name,Diet,Country,Height,weight
233,shanag,0,Mongolia,0.2385,1.0


In [12]:
dinotest[dinotest['name'] == 'indosuchus']

,name,Diet,Country,Height,weight
128,indosuchus,0,India,3.71,2360.539442
